In [1]:
import pandas as pd

file1 = pd.read_excel('data/附件.xlsx',sheet_name='表单1')
file2 = pd.read_excel('data/附件.xlsx',sheet_name='表单2')
file3 = pd.read_excel('data/附件.xlsx',sheet_name='表单3')

# 1 问题三

In [2]:
from tqdm import tqdm
import re
tqdm.pandas()

def clear_id(s):
    ss = str(s)
    n = int(re.findall(r"\d+",ss)[0])
    return n
file2['id'] = file2['文物采样点'].progress_apply(clear_id)
file1['id'] = file1['文物编号'].progress_apply(clear_id)


100%|███████████████████████████████████████████████████████████████████████████████| 58/58 [00:00<00:00, 58156.74it/s]


In [3]:
cols = ['表面风化', '二氧化硅(SiO2)',
       '氧化钠(Na2O)', '氧化钾(K2O)', '氧化钙(CaO)', '氧化镁(MgO)', '氧化铝(Al2O3)',
       '氧化铁(Fe2O3)', '氧化铜(CuO)', '氧化铅(PbO)', '氧化钡(BaO)', '五氧化二磷(P2O5)',
       '氧化锶(SrO)', '氧化锡(SnO2)', '二氧化硫(SO2)']


In [4]:
file_data = pd.merge(file1,file2,on='id')
Gao_data = file_data[file_data['类型']=='高钾'][cols]
Qian_data = file_data[file_data['类型']=='铅钡'][cols]
test_all = file3[cols]


In [5]:
Qian_data = Qian_data.fillna(0)
Gao_data = Gao_data.fillna(0)
test_all = test_all.fillna(0)

In [6]:
train_all = pd.concat([Gao_data,Qian_data])
encode_data = pd.concat([train_all,test_all])
len(train_all)

69

In [7]:
from sklearn.preprocessing import LabelEncoder

all_data = pd.DataFrame() # 用于存放编码后的训练数据
label = LabelEncoder() # 标号编码器
X= encode_data
for c in X.columns: # 对每个特征列
    if X[c].dtype=='object': # 如果是字符串形式的(字符串读到pandas里dtype是object)
        all_data[c] = label.fit_transform(X[c]) # 将整个这一列进行标号编码，写到新的dataframe里
    else: # 其它类型的特征(数值,布尔)保持原样写入
        all_data[c] = list(X[c])


In [8]:
train = all_data[0:len(train_all)]
test = all_data[len(train_all):]

In [9]:
from sklearn.model_selection import train_test_split # 拆分数据
import numpy as np
X_var,y_var= train,[0]*len(Gao_data)+[1]*len(Qian_data)
X_train, X_test, y_train, y_test = train_test_split(X_var, y_var, test_size = 0.2, random_state = 123)


## 1.2 建立决策树

In [18]:
from sklearn.tree import DecisionTreeClassifier as dtc # 树算法
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
model = dtc(criterion = 'entropy', max_depth = 4)
model.fit(X_train, y_train)
pred_model = model.predict(X_test)
print(accuracy_score(y_test, pred_model))

1.0


In [13]:
precision_score(y_test, pred_model)

1.0

In [20]:
recall_score(y_test, pred_model)

1.0

In [19]:
f1_score(y_test, pred_model, labels=None, pos_label=1)

1.0

In [22]:
from sklearn.metrics import precision_recall_curve, average_precision_score
# 求precision和recall值
precision, recall, _ = precision_recall_curve(y_test, pred_model)
#求AP值
print('AP值：{:.3f}'.format(average_precision_score(y_test, pred_model)))

AP值：1.000


In [ ]:
from sklearn.tree import plot_tree # 树图
import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus']=False
plt.rcParams['font.sans-serif'] = ['SimHei']
model = dtc(criterion = 'entropy', max_depth = 4)
model.fit(X_var, y_var)
feature_names = X_var.columns
target_names = ['高钾玻璃','铅钡玻璃']
 
plot_tree(model, 
          feature_names = feature_names, 
          class_names = target_names, 
          filled = False, 
          rounded = True)
 
plt.savefig('./img3/决策树可视化.png')
plt.show()

In [ ]:
from sklearn.tree import plot_tree # 树图
import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus']=False
plt.rcParams['font.sans-serif'] = ['SimHei']
model = dtc(criterion = 'gini', max_depth = 4)
model.fit(X_var, y_var)
feature_names = X_var.columns
target_names = ['高钾玻璃','铅钡玻璃']
 
plot_tree(model, 
          feature_names = feature_names, 
          class_names = target_names)
plt.show()

In [ ]:
model.predict(test)

In [ ]:
# 预测结果为：高钾,铅钡,铅钡,铅钡,铅钡,高钾,高钾,铅钡
model.predict(test)

## 2.3 判断亚类

In [ ]:
from sklearn.cluster import AgglomerativeClustering #导入sklearn的层次聚类函数
df1 = all_data[0:len(Gao_data)]
data1 = np.array(df1)
gao_test = test.iloc[[0,5,6]]
clf1 = AgglomerativeClustering(n_clusters = 3, linkage = 'ward')
s = clf1.fit(data1)
pred1 = clf1.fit_predict(gao_test)

In [ ]:
pred1

In [ ]:
from sklearn.cluster import AgglomerativeClustering #导入sklearn的层次聚类函数
df1 = all_data[len(Gao_data):len(Gao_data)+len(Qian_data)]
data1 = np.array(df1)
gao_test = test.iloc[[1,2,3,4,7]]
clf1 = AgglomerativeClustering(n_clusters = 3, linkage = 'ward')
s = clf1.fit(data1)
pred1 = clf1.fit_predict(gao_test)
pred1